In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import time

In [3]:
from IPython.display import HTML

In [4]:
from brightness_controller import BrightnessController
from controller import Controller
from mpv import MPVController
from radar import LD2450
from radar_data_collector import RadarDataCollector
from sliding_average import SlidingAverage

In [143]:
uartdev = "/dev/ttyS0"

## Sliding average

In [6]:
SlidingAverage.show_check(n=12, ws=4)

  60 |   60    0    0    0 |   60    0    0    0 |  15.00  15.00 | yes
 -83 |   60  -83    0    0 |   60  -83    0    0 |  -5.75  -5.75 | yes
  99 |   60  -83   99    0 |   60  -83   99    0 |  19.00  19.00 | yes
  76 |   60  -83   99   76 |   60  -83   99   76 |  38.00  38.00 | yes
 -44 |  -83   99   76  -44 |  -44  -83   99   76 |  12.00  12.00 | yes
 -80 |   99   76  -44  -80 |  -44  -80   99   76 |  12.75  12.75 | yes
 -16 |   76  -44  -80  -16 |  -44  -80  -16   76 | -16.00 -16.00 | yes
  85 |  -44  -80  -16   85 |  -44  -80  -16   85 | -13.75 -13.75 | yes
   2 |  -80  -16   85    2 |    2  -80  -16   85 |  -2.25  -2.25 | yes
  10 |  -16   85    2   10 |    2   10  -16   85 |  20.25  20.25 | yes
  98 |   85    2   10   98 |    2   10   98   85 |  48.75  48.75 | yes
  32 |    2   10   98   32 |    2   10   98   32 |  35.50  35.50 | yes


In [7]:
avg = SlidingAverage(4)
for i in range(10):
    avg.add(i)
    print(f"{i} | {avg.vals} | {avg.avg:.2f}")

0 | [0, 0, 0, 0] | 0.00
1 | [0, 1, 0, 0] | 0.25
2 | [0, 1, 2, 0] | 0.75
3 | [0, 1, 2, 3] | 1.50
4 | [4, 1, 2, 3] | 2.50
5 | [4, 5, 2, 3] | 3.50
6 | [4, 5, 6, 3] | 4.50
7 | [4, 5, 6, 7] | 5.50
8 | [8, 5, 6, 7] | 6.50
9 | [8, 9, 6, 7] | 7.50


## Radar LD2450

In [8]:
r = LD2450(uartdev)

In [9]:
r.uartdev

'/dev/ttyS0'

In [10]:
r.verbose

False

In [11]:
print(r.in_waiting)
r.clean()
print(r.in_waiting)

780
0


In [12]:
r.restart()

In [13]:
r.get_firmware_version()

'V2.04.23101915'

In [14]:
r.get_mac_address()

'B8 06 B8 19 8E BA'

In [15]:
r.get_bluetooth_state()

True

In [16]:
r.get_tracking_mode()

2

In [17]:
r.get_zone_filtering()

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [19]:
r.show_info()

UART device: /dev/ttyS0
Firmware version: V2.04.23101915
Bluetooth: ON (MAC address: B8 06 B8 19 8E BA)
Multi tracking: ON
Zone filtering: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [20]:
r.set_bluetooth_on(restart=True)
r.set_multi_tracking()

In [21]:
r.show_info()

UART device: /dev/ttyS0
Firmware version: V2.04.23101915
Bluetooth: ON (MAC address: B8 06 B8 19 8E BA)
Multi tracking: ON
Zone filtering: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [22]:
r.test(100)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99

Firmware version: ['V2.04.23101915']
MAC address: ['B8 06 B8 19 8E BA']
Tracking mode: [2]


In [23]:
print(r.in_waiting)
r.show_data(50)

1050
    0     0 |     0     0 |     0     0 | IN:     0 | sample:     1
    0     0 |     0     0 |     0     0 | IN:     0 | sample:     2
    0     0 |     0     0 |     0     0 | IN:     0 | sample:     3
    0     0 |     0     0 |     0     0 | IN:     0 | sample:     4
    0     0 |     0     0 |     0     0 | IN:     0 | sample:     5
    0     0 |     0     0 |     0     0 | IN:     0 | sample:     6
    0     0 |     0     0 |     0     0 | IN:     0 | sample:     7
    0     0 |     0     0 |     0     0 | IN:     0 | sample:     8
    0     0 |     0     0 |     0     0 | IN:     0 | sample:     9
    0     0 |     0     0 |     0     0 | IN:     0 | sample:    10
    0     0 |     0     0 |     0     0 | IN:     0 | sample:    11

    0     0 |     0     0 |     0     0 | IN:     0 | sample:     1
    0     0 |     0     0 |     0     0 | IN:     0 | sample:     2
    0     0 |     0     0 |     0     0 | IN:     0 | sample:     3
    0     0 |     0     0 |     0     0 | 

## Radar 2410C

In [24]:
# ...

## Radar 2461

In [25]:
# ...

## Radar data collector

In [127]:
rdc = RadarDataCollector(uartdev)

In [128]:
rdc.radar.show_info()

UART device: /dev/ttyS0
Firmware version: V2.04.23101915
Bluetooth: ON (MAC address: B8 06 B8 19 8E BA)
Multi tracking: ON
Zone filtering: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [129]:
rdc.uartdev

'/dev/ttyS0'

In [130]:
rdc.default_bluetooth

True

In [131]:
rdc.default_multi_tracking

True

In [132]:
rdc.active

True

In [133]:
rdc.qsize

0

In [134]:
rdc.empty

True

In [135]:
rdc.start()

Radar UP.


In [137]:
for _ in range(10):
    print(rdc.qsize)
    time.sleep(1)

118
130
141
152
163
175
186
197
208
220


In [138]:
rdc.active = False

Radar DOWN.


In [139]:
for _ in range(10):
    print(rdc.qsize)
    time.sleep(1)

270
270
270
270
270
270
270
270
270
270


## Camera

In [140]:
# ...

## Direct brightness

In [141]:
i2c_num = 2
br_ctl = BrightnessController(i2c_num)

In [142]:
for i in range(0, 101):
    br_ctl.set(i)
    time.sleep(0.01)

FileNotFoundError: [Errno 2] No such file or directory: '/dev/i2c-2'

## Manipulating gamma

In [1]:
# ...

## MPV

In [6]:
mpv_ctl = MPVController()

In [7]:
# mpv_ctl.get_property("working-directory")

In [8]:
# mpv_ctl.get_property("term-status-msg")

In [9]:
# mpv_ctl.socket.recv(1024)

In [10]:
# problems = []
# for i in range(500):
#     res = mpv_ctl.load_file(video_fpath)
#     if res is not None and res['error'] == 'success':
#         continue

#     problems.append(i)

# print(problems)

In [142]:
mpv_ctl.set_property("loop-file", "inf")

{'request_id': 902, 'error': 'success'}

In [21]:
mpv_ctl.clear()

{'data': None, 'request_id': 1, 'error': 'success'}

In [11]:
mpv_ctl.load_file(video_fpath)

{'data': {'playlist_entry_id': 1}, 'request_id': 1, 'error': 'success'}

In [14]:
mpv_ctl.get_property("duration")

{'data': 888.48, 'request_id': 4, 'error': 'success'}

In [15]:
mpv_ctl.get_property("playlist/count")

{'data': 1, 'request_id': 5, 'error': 'success'}

In [16]:
mpv_ctl.get_property("playlist/0/filename")

{'data': '/home/tami/videos/head_on_floor_1920x1080.mp4',
 'request_id': 6,
 'error': 'success'}

In [17]:
mpv_ctl.play()

{'request_id': 7, 'error': 'success'}

In [18]:
mpv_ctl.get_property("pause")

{'data': False, 'request_id': 8, 'error': 'success'}

In [12]:
mpv_ctl.pause()

{'request_id': 2, 'error': 'success'}

In [140]:
mpv_ctl.get_property("pause")

{'data': False, 'request_id': 900, 'error': 'success'}

In [139]:
mpv_ctl.get_property("time-pos")

{'data': 4.541583, 'request_id': 899, 'error': 'success'}

In [75]:
mpv_ctl.get_property("playback-time")

{'data': -0.004625, 'request_id': 634, 'error': 'success'}

In [138]:
mpv_ctl.get_property("time-remaining")

{'data': 9.893542, 'request_id': 898, 'error': 'success'}

In [137]:
mpv_ctl.get_property("framedrop")

{'data': 'vo', 'request_id': 897, 'error': 'success'}

In [136]:
mpv_ctl.get_property("frame-drop-count")

{'data': 4, 'request_id': 896, 'error': 'success'}

In [113]:
mpv_ctl.seek(0)

{'data': None, 'request_id': 873, 'error': 'success'}

In [116]:
mpv_ctl.get_property("playback-time")

{'data': 7.87825, 'request_id': 876, 'error': 'success'}

In [110]:
mpv_ctl.get_property("time-remaining")

{'data': 4.012667, 'request_id': 870, 'error': 'success'}

In [145]:
for i in range(-100, 1):
    mpv_ctl.set_brightness(i, osd=True)
    time.sleep(0.1)

In [153]:
mpv_ctl.get_property("volume")

{'data': 100.0, 'request_id': 115, 'error': 'success'}

## Controller

In [ ]:
ws = 1
dmin = 1000
dmax = 2500

ctl = Controller(uartdev, ws, dmin, dmax, screen_bus=i2c_num)

In [ ]:
ctl.start()

## Plotter

In [ ]:
p = Plotter(uartdev, ws, dmin, dmax, verbose=True, xmin=-3000, xmax=3000, ymin=-4000)

In [ ]:
ani = p.start()

In [ ]:
HTML(ani.to_jshtml())